# slurm cluster testing
the following is some testing for writing and executing sbatch jobs for data in gcs in the new slurm cluster

In [2]:
import pandas as pd

In [ ]:
# setup for jupyter in slurm
# https://towardsdatascience.com/running-jupyter-notebook-in-google-cloud-platform-in-15-min-61e16da34d52

# copy key to slurm cluster
#gcloud compute scp keys/genotools-ee5d11817508.json --zone "us-central1-a" "genoslurm-login-7vbw7cih-001":/home/dan_datatecnica_com/keys/  --tunnel-through-iap --project "genotools"


# use the following command in interactive controller to mount gcs bucket to slurm controller
# gcsfuse --dir-mode 776 --file-mode 776 --implicit-dirs --key-file=/home/dan_datatecnica_com/keys/genotools-ee5d11817508.json gp2_data /home/dan_datatecnica_com/data

In [1]:
!gsutil ls gs://gp2_uk/gp2_idats/ > tmp/idats.txt

In [15]:
idats_in = pd.read_csv('tmp/idats.txt', header=None)
idat_list = idats_in.loc[1:,0]
idat_paths = [x.replace('gs://gp2_data/gp2_idats','/home/dan_datatecnica_com/data/gp2_idats')[:-1] for x in idat_list]
idat_paths



['/home/dan_datatecnica_com/data/gp2_idats/204620380001']

In [7]:
# with open(f'tmp/call_gts.sbatch', 'w') as f:

for idat in idat_list:
    code = idat.split('/')[-2]
    idat_path = '/home/dan_datatecnica_com/data/gp2_idats'
#         idat_to_ped_cmd = f'\
# {iaap} gencall \
# {bpm} \
# {egt} \
# {plink_file_path} \
# -f {idat_path}/{code} \
# -p \
# -t 8'

204620380001


In [18]:
# file_paths = ['file1.txt', 'file2.txt', 'file3.txt']
# command = 'gencall -f {file_path}'
datapath = '/home/dan_datatecnica_com/data'
iaap = '/home/dan_datatecnica_com/GenoTools/executables/iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'
bpm = f'{datapath}/ilmn_files/NeuroBooster_20042459_A1.bpm'
egt = f'{datapath}/ilmn_files/recluster_09272022.egt'
plink_file_path = f'{datapath}/ilmn_files/gp2_plink'



# Create a sbatch script
with open('tmp/call_gts.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    f.write('SBATCH --job-name=callgts\n')
    f.write('SBATCH --output=callgts.out\n')
    f.write('SBATCH --error=callgts.err\n')
    f.write(f'SBATCH --ntasks={len(idat_paths)}\n')
    f.write('SBATCH --time=01:00:00\n')
    f.write(f'SBATCH --array=1-{len(idat_paths)}\n')
    f.write('\n')
    f.write('echo "Starting job"\n')
    f.write('date\n')
    f.write('\n')
    for i, idat_path in enumerate(idat_paths):
        code = idat_path.split('/')[-1]
        idat_dir_in = f'{datapath}/gp2_idats/{code}'
        command = f'\
{iaap} gencall \
{bpm} \
{egt} \
{plink_file_path} \
-f {idat_dir_in} \
-p \
-t 8'
        f.write(f'# Run command for file {idat_dir_in}\n')
        f.write(f'{command}\n')
    f.write('\n')
    f.write('echo "Job finished"\n')
    f.write('date\n')
f.close()


In [1]:
!gcloud compute scp tmp/call_gts.sh --zone "us-central1-a" "genoslurm-login-7vbw7cih-001":/home/dan_datatecnica_com/sbatch_scripts/ --project "genotools"

External IP address was not found; defaulting to using IAP tunneling.
/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
call_gts.sh                                   100%  680    21.1KB/s   00:00    


In [21]:
!ls tmp

call_gts.sh idats.txt   testjob.sh
